In [227]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression


In [254]:

import warnings
warnings.filterwarnings("ignore")

In [228]:
#nltk.download('stopwords')

In [99]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
wordCount = defaultdict(int)
punctuation = set(string.punctuation)

In [100]:
#print(stop_words)

{'s', 'some', 'shan', 've', 'nor', 't', 'these', 'what', 'out', 'i', "wasn't", "hadn't", 'do', 'at', "didn't", 'both', 'because', 'didn', 'under', 'wasn', 'we', 'be', 'of', 'again', 'same', "wouldn't", 'am', "that'll", 'wouldn', 'the', 'on', 'myself', 'himself', 'needn', 'further', 'about', 'theirs', 'him', 'don', "should've", 'your', 'when', 'were', "mightn't", 'up', 'won', 'or', 'any', 'those', 'ma', 'for', 'above', "aren't", 'which', 'a', "haven't", "mustn't", 'hadn', 'should', 'is', 'weren', 'to', 'few', 'too', 'before', 'it', "don't", 'shouldn', 'our', 'hasn', 'ours', 'who', 'where', 'most', 'he', 'will', 'this', 'into', 'being', 'y', "hasn't", "doesn't", 'hers', 'have', 'as', 'ourselves', 'she', 'has', 'yourself', "couldn't", 'herself', 'aren', "you've", 'having', 'with', 'between', 'ain', "you'll", 'isn', 'doesn', 'against', 'how', 'yourselves', 'here', 'whom', 'you', 'down', 'such', 'yours', 'doing', 'mustn', 'not', 'why', 'all', 'that', 'more', 'll', "needn't", 'so', 'from', '

In [94]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [95]:

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [168]:

data = []
for d in readGz("train_Category.json.gz"):
    data.append(d)

In [169]:

dataTrain = data[:90000]
dataValid = data[90000:100000]
print(len(dataTrain))
print(len(dataValid))

90000
10000


In [109]:
punctuation = set(string.punctuation)

In [107]:
wordCount1 = defaultdict(int)
for d in dataTrain:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split(): 
            wordCount1[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words = [x[1] for x in counts2[:1000]]

267585

In [110]:
wordCount2 = defaultdict(int)
for d in dataTrain:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w2 in r.split(): 
        if w2 in [d for d in stop_words]: 
            continue
        else:
            wordCount2[w2] += 1

counts2 = [(wordCount2[w], w) for w in wordCount2]
counts2.sort()
counts2.reverse()
words2 = [x[1] for x in counts2[:1000]]


267438

In [111]:
wordCount3 = defaultdict(int)
for d in dataTrain:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w2 in r.split(): 
        w3 = stemmer.stem(w2)
        if w3 in [d for d in stop_words]: 
            continue
        else:
            wordCount3[w3] += 1

len(wordCount3)

220609

In [262]:
counts3 = [(wordCount3[w], w) for w in wordCount3]
counts3.sort()
counts3.reverse()

In [178]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [137]:
wordId2 = dict(zip(words2, range(len(words2))))
wordSet2 = set(words2)

In [336]:
words3 = [x[1] for x in counts3[:50000]]
wordId3 = dict(zip(words3, range(len(words3))))
wordSet3 = set(words3)

In [337]:
len(words3)

50000

In [338]:
def feature(datum): 
    feat = [0]*len(words3)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        w2 = stemmer.stem(w)
        if w2 not in wordSet3: continue
        if w2 in [d for d in stop_words]: continue
        feat[wordId3[w2]] += 1
    
    return feat

In [ ]:
X = [feature(d) for d in data]

In [ ]:
y = [d['genreID'] for d in data]

In [ ]:
len(X)

In [285]:
mod = LogisticRegression(solver='lbfgs', C=10, multi_class='auto', max_iter=1000)
mod1 = linear_model.LogisticRegression(C = 0.15)
mod2 = linear_model.LogisticRegression(C = 1)
mod.fit(Xtrain,ytrain)
mod1.fit(Xtrain,ytrain)
mod2.fit(Xtrain,ytrain)

LogisticRegression(C=1)

In [ ]:
mod1 = linear_model.LogisticRegression(C = 0.2)
#0.7366  0.3 8000
#0.7378  0.2 8000
#0.7482  0.2 14000
mod1.fit(X,y)

In [ ]:
mod2 = linear_model.LogisticRegression(C = 0.13)
mod2.fit(X,y)
#0.7392  8000
#0.7492 14000

In [328]:
mod3 = linear_model.LogisticRegression(C = 0.1)
mod3.fit(X,y)

LogisticRegression(C=0.1)

In [317]:
mod22 = linear_model.LogisticRegression(C=0.3, class_weight='balanced')
mod22.fit(X,y)
#  8000 0.7138

LogisticRegression(C=0.3, class_weight='balanced')

In [296]:
catDict = {
  "children": 0,
  "comics_graphic": 1,
  "fantasy_paranormal": 2,
  "mystery_thriller_crime": 3,
  "young_adult": 4
}

In [ ]:
predictions = open("predictions_Category.csv", 'w')
predictions.write("userID,reviewID,prediction\n")
for l in readGz("test_Category.json.gz"):
    #cat = mod.predict([feature(l)])
    cat = mod1.predict([feature(l)])
    #cat = mod2.predict([feature(l)])
    #cat = mod3.predict([feature(l)])
    #cat = mod4.predict([feature(l)])
    #cat = mod5.predict([feature(l)])
    #cat = mod6.predict([feature(l)])
    predictions.write(l['user_id'] + ',' + l['review_id'] + "," + str(cat[0]) + "\n")

predictions.close()